In [156]:
import pandas as pd
pd.set_option('display.max_columns', None)

# TEST


In [157]:
import pandas as pd

# Cargar el DataFrame
columnas = ['user_id', 'item_id', 'recommend', 'app_name', 'Sentiment_analysis', 'playtime_forever']
df = pd.read_csv('data_fusionada.csv', usecols=columnas)

# Convertir 'user_id' a categórico y crear una nueva columna con los códigos
df['user_id_code'] = pd.Categorical(df['user_id']).codes

# Ahora df tiene tanto 'user_id' como 'user_id_code'
df.head()

,app_name,item_id,playtime_forever,user_id,recommend,Sentiment_analysis,user_id_code
0,Carmageddon Max Pack,282010,466.0,InstigatorAU,True,1.0,10321
1,Half-Life,70,1395.0,EizanAratoFujimaki,True,2.0,9579
2,Half-Life,70,590.0,GamerFag,True,0.0,9910
3,Half-Life,70,5599.0,76561198020928326,True,2.0,1069
4,Half-Life,70,64.0,Bluegills,True,2.0,8887


In [158]:
df['recommend1'] = 1
df

,app_name,item_id,playtime_forever,user_id,recommend,Sentiment_analysis,user_id_code,recommend1
0,Carmageddon Max Pack,282010,466.0,InstigatorAU,True,1.0,10321,1
1,Half-Life,70,1395.0,EizanAratoFujimaki,True,2.0,9579,1
2,Half-Life,70,590.0,GamerFag,True,0.0,9910,1
3,Half-Life,70,5599.0,76561198020928326,True,2.0,1069,1
4,Half-Life,70,64.0,Bluegills,True,2.0,8887,1
...,...,...,...,...,...,...,...,...
40010,Counter-Strike: Condition Zero,80,2416.0,76561198015050660,False,1.0,950,1
40011,Counter-Strike: Condition Zero,80,15.0,76561198023508728,False,1.0,1125,1
40012,Counter-Strike: Condition Zero,80,442.0,green290,True,1.0,15160,1
40013,Counter-Strike: Condition Zero,80,72.0,174gamecuman700kngkakak,True,1.0,127,1


In [159]:
df['playtime_forever'] = df['playtime_forever'].astype(int)

In [160]:
# Agrupar y sumar
resultados = df.groupby('user_id').agg({
    'playtime_forever': 'sum',
    'recommend1': 'sum'
}).reset_index()

In [161]:
resultados

,user_id,playtime_forever,recommend1
0,--000--,2949,1
1,--ace--,1269,1
2,--ionex--,36320,2
3,-2SV-vuLB-Kg,31337,3
4,-Azsael-,11889,1
...,...,...,...
18951,zuzuga2003,72732,4
18952,zv_odd,6452,8
18953,zvanik,38959,3
18954,zynxgameth,3846,1


In [162]:
# # Ordenar por la columna 'playtime_forever' de manera ascendente
# df_sorted = resultados.sort_values(by='playtime_forever')

# # Mostrar el DataFrame ordenado
# df_sorted.head(50)

In [163]:
df_filtro_time = resultados[resultados['playtime_forever'] >= 3000]
df_filtro_time

,user_id,playtime_forever,recommend1
2,--ionex--,36320,2
3,-2SV-vuLB-Kg,31337,3
4,-Azsael-,11889,1
6,-Kenny,12842,1
7,-Mad-,3460,2
...,...,...,...
18951,zuzuga2003,72732,4
18952,zv_odd,6452,8
18953,zvanik,38959,3
18954,zynxgameth,3846,1


In [164]:
df_filtro_rec = df_filtro_time[df_filtro_time['recommend1'] >= 3]
df_filtro_rec

,user_id,playtime_forever,recommend1
3,-2SV-vuLB-Kg,31337,3
9,-SEVEN-,187309,9
17,00000000000000000001227,50964,4
23,01001000-01101001,13243,3
26,01189958889189157253,34740,4
...,...,...,...
18945,zp3413,23704,3
18951,zuzuga2003,72732,4
18952,zv_odd,6452,8
18953,zvanik,38959,3


In [165]:
columnas = ['user_id', 'item_id', 'recommend', 'app_name', 'Sentiment_analysis', 'playtime_forever']
df_main = pd.read_csv('data_fusionada.csv', usecols=columnas)

In [166]:
# Supongamos que 'user_id' es la columna que contiene los identificadores de usuario en ambos DataFrames
usuarios_filtrados = df_main[df_main['user_id'].isin(df_filtro_rec['user_id'])]

# Ahora, 'usuarios_filtrados' contiene solo las filas de 'df_main' que corresponden a los 4981 usuarios en 'df_nuevo'
usuarios_filtrados

,app_name,item_id,playtime_forever,user_id,recommend,Sentiment_analysis
2,Half-Life,70,590.0,GamerFag,True,0.0
4,Half-Life,70,64.0,Bluegills,True,2.0
5,Half-Life,70,3235.0,76561198071955492,True,2.0
6,Half-Life,70,1301.0,digfernandes,True,0.0
7,Half-Life,70,21.0,76561197970812298,True,2.0
...,...,...,...,...,...,...
40009,Counter-Strike: Condition Zero,80,3721.0,KILLERamateur,True,1.0
40010,Counter-Strike: Condition Zero,80,2416.0,76561198015050660,False,1.0
40012,Counter-Strike: Condition Zero,80,442.0,green290,True,1.0
40013,Counter-Strike: Condition Zero,80,72.0,174gamecuman700kngkakak,True,1.0


In [173]:
df_main

,app_name,item_id,playtime_forever,user_id,recommend,Sentiment_analysis
0,Carmageddon Max Pack,282010,466.0,InstigatorAU,True,1.0
1,Half-Life,70,1395.0,EizanAratoFujimaki,True,2.0
2,Half-Life,70,590.0,GamerFag,True,0.0
3,Half-Life,70,5599.0,76561198020928326,True,2.0
4,Half-Life,70,64.0,Bluegills,True,2.0
...,...,...,...,...,...,...
40010,Counter-Strike: Condition Zero,80,2416.0,76561198015050660,False,1.0
40011,Counter-Strike: Condition Zero,80,15.0,76561198023508728,False,1.0
40012,Counter-Strike: Condition Zero,80,442.0,green290,True,1.0
40013,Counter-Strike: Condition Zero,80,72.0,174gamecuman700kngkakak,True,1.0


In [172]:
usuarios_filtrados['Sentiment_analysis'].unique()

array([0, 2, 1])

In [167]:
usuarios_filtrados['Sentiment_analysis'] = usuarios_filtrados['Sentiment_analysis'].astype(int)

C:\Users\Isaac\AppData\Local\Temp\ipykernel_12948\696766697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios_filtrados['Sentiment_analysis'] = usuarios_filtrados['Sentiment_analysis'].astype(int)


In [168]:
# Función para visualizar la calidad de nuestros datos

def resumen_tipos_de_datos(df):
    
    resumen = []

    for col in df.columns:

        tipos_presentes = df[col].apply(lambda x: type(x).__name__).unique()
        cantidad_nulos = df[col].isnull().sum()
        resumen.append({
            'Columna': col,
            'Tipos de Dato': tipos_presentes.tolist(),
            'Cantidad de Nulos': cantidad_nulos
        })   
    return pd.DataFrame(resumen)

In [169]:
resumen_tipos_de_datos(usuarios_filtrados)

,Columna,Tipos de Dato,Cantidad de Nulos
0,app_name,[str],0
1,item_id,[int],0
2,playtime_forever,[float],0
3,user_id,[str],0
4,recommend,[bool],0
5,Sentiment_analysis,[int],0


In [170]:
# Guardar el DataFrame reducido en un nuevo archivo CSV
usuarios_filtrados.to_csv('usuarios_filtrados.csv', index=False)